### Some thing went wrong

In [7]:
training_data = [
    ['Green', 3, 'Mango'],
    ['Wellow', 3, 'Mango'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'] 
]

header = ['color','diameter','label']

def unique_values(rows, col):
    # Fin the unique values for a columns  in dataset
    return set([rows[col] for row in rows])

def class_counts(rows):
    # COunt the number of each type of example in a dataset
    counts = {} # dictionary of label -> counts
    for row in rows:
        label = row[-1]
        if(label not in counts):
            counts[label] = 0
        counts[label] += 1
    return counts

def is_numerics(value):
    return is_numerics(value=value)






In [8]:
class Question:
    '''A question is used to partition a dataset'''
     
    def __init__(self,column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the 
        # feature value in this question
        val = example[self.column]
        if is_numerics(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
        # Helper method to print
        condition = "=="
        if is_numerics(self.value):
            condition = ">="
        return f"Is {self.column} {condition} {str(self.value)}"

def paritition(rows, question):
    '''For each row in the dataset, check if it matches the question.
    If so, add it to "true row", otherwise, add it to "flase row". '''
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def gini(rows):
    """Calculate the Gini Impurity for a list of rows"""
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        pro_of_lbl = counts[lbl] / float(len(rows))
        impurity -= pro_of_lbl**2
    return impurity

# information gain
def info_gain(left, right, current_undercertainly):
    p = float(len(left)) / (len(left) + len(right))
    return current_undercertainly - p*gini(left) - (1-p) * gini(right)


In [9]:
def find_best_split(rows):
    best_gain = 0 ## keep track of the best information gain
    best_question = None # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 # number of columns

    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            # try splititing  the dataset
            true_rows, false_row = paritition(rows, val)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_row) == 0:
                continue
            gain = info_gain(true_rows, false_row, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


In [10]:
class Leaf:
    # A leaf node classifies data
    def __init__(self,rows):
        self.predictions = class_counts(rows)
    
class Decision_Node: 
    def ____init__(self,
                    question,
                    true_branch,
                    false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_brannch = false_branch

def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = paritition(rows, question)

    # Recursively build the true branch 
    true_branch = build_tree(true_rows)

    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [11]:
def print_tree(node, spacing = ""):
    if isinstance(node, Leaf):
        print(spacing + "Predict: ", node.predictions)
        return
    print(spacing+ str(node.question))
    ## Call thi function recursive
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + " ")
    print(spacing + "--> False: ")
    print_tree(node.false_branch, spacing + " ")    

In [12]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

def print_leaf(counts):
    """Print the predictions at a Leaf."""
    total = sum(counts.value())*1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl]/total * 100)) + "%"
    return probs

In [14]:
if __name__ == 'main':
    my_tree = build_tree(training_data)
    print_tree(my_tree)
# Evalue
testing_data = [
    ['Green', 3, 'Lemon'],
    ['Wellow', 3, 'Mango'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'] 
]
for row in testing_data:
    print(f"Actual: {row[-1]}. Predicted: {print_leaf(classify(row, my_tree))}")

NameError: name 'my_tree' is not defined